In [1]:
import numpy as np
from functools import partial

In [2]:
# 随机生成 D 个长度为 N 的时间序列，用于测试
def gen_rand_data(D, N, mean=1000, std=5, seed=None): 
    if seed is not None:
        np.random.seed(seed)
    data = np.random.normal(loc=mean, scale=std, size=(D, N))
    return data

# 从时间序列获得漂移率、波动率、相关系数
def get_parameters(data, dt=1.0):
    ret_data = data[:,1:]/data[:,:-1]-1
    mu = np.mean(ret_data, axis=1)/dt
    sigma = np.std(ret_data, axis=1, ddof=1)/dt
    rho = np.corrcoef(ret_data)
    return mu, sigma, rho

def max_call_option(S, K):
    return max(np.max(S)-K,0)

In [3]:
import yfinance as yf

tickers_train = ['MSFT', 'GOOGL', 'AMZN', 'AAPL']
start_date, end_date = "2018-01-01", "2023-12-31"
stock_data_train = []
for ticker in tickers_train:
    ticker_data = yf.Ticker(ticker).history(start=start_date, end=end_date)['Close']
    ticker_data = ticker_data/np.mean(ticker_data)*1000
    stock_data_train.append(ticker_data)
stock_data_train = np.array(stock_data_train)
D = stock_data_train.shape[0]
T, N, M = 1.0, 1000, 10000
dt = T/N

s0 = stock_data_train[:,0]
mu, sigma, rho = get_parameters(stock_data_train, dt)
r = 0.04 / N
option_func = partial(max_call_option, K = 1000)

用真实的股票数据来构造max option：

In [8]:
import numpy as np
import torch
from torch import nn, optim, sigmoid, softmax

from delta_hedge import run_dual, set_seed
from transformer.model_percentiles import OptionTransformer


def train_real_dual(option, r, config, num_run, real_S, mu, sigma, rho):
    set_seed()

    # Monte Carlo config
    D = config["n_assets"]
    M = config["n_mcmc"]
    N = config["n_timesteps"]

    # Model config
    d_model = config["d_model"]
    n_layers = config["n_layers"]
    n_head = config["n_head"]
    dropout = config["dropout"]

    model1 = OptionTransformer(d_model, n_layers, n_head, D, M, dropout).to(config["device"])
    model2 = OptionTransformer(d_model, n_layers, n_head, D, M, dropout).to(config["device"])

    optimizer1 = optim.Adam(model1.parameters(), lr=config["lr"])
    optimizer2 = optim.Adam(model1.parameters(), lr=config["lr"])

    real_S = torch.tensor(real_S).float().to(config["device"])

    for n in range(num_run):
        T = 1.0  # the absolute value for `T` is really unnecessary for us

        # run once till the maturity, give back the premium
        alpha_1, alpha_2 = run_dual(model1, model2, option, real_S, mu, sigma, rho, T, N, M, r)
        print(f"Trial {n}--Seller Return:{alpha_1:6f}, Buyer Return:{alpha_2:6f}")

        # compute loss
        criterion = nn.MSELoss().to(config["device"])
        alpha = torch.min(torch.stack((alpha_1, alpha_2)))
        loss = criterion(torch.sigmoid(alpha), torch.tensor(1.0))

        # optimize model
        optimizer1.zero_grad()
        optimizer2.zero_grad()
        loss.backward()
        optimizer1.step()
        optimizer2.step()
    

In [9]:
config = {
    "n_assets": 4,
    "n_mcmc": 100,
    "n_timesteps": 100,
    "d_model": 20,
    "n_layers": 5,
    "n_head": 10,
    "dropout": 0.1,
    "lr": 1e-3,
    "num_run": 10,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu")
}

train_real_dual(option_func, r, config, config["num_run"], stock_data_train, mu, sigma, rho)

Sample 100 is added.
Dataset built.
Start--Seller:-1.938569, Buyer:2.070111
Price:0.7811535596847534
End--Seller:0.560036, Buyer:-0.476604
Trial 0--Seller Return:1.288891, Buyer Return:-1.230231
Sample 100 is added.
Dataset built.
Start--Seller:-2.262041, Buyer:2.272485
Price:0.48619621992111206
End--Seller:0.449060, Buyer:-0.511730
Trial 1--Seller Return:1.198520, Buyer Return:-1.225185
Sample 100 is added.
Dataset built.
Start--Seller:-2.723418, Buyer:2.237691
Price:0.4054524004459381
End--Seller:0.093739, Buyer:-0.546722
Trial 2--Seller Return:1.034419, Buyer Return:-1.244324
Sample 100 is added.
Dataset built.
Start--Seller:-2.708753, Buyer:2.434329
Price:0.35262802243232727
End--Seller:0.244352, Buyer:-0.098523
Trial 3--Seller Return:1.090208, Buyer Return:-1.040472
Sample 100 is added.
Dataset built.
Start--Seller:-2.684824, Buyer:2.492861
Price:0.39437806606292725
End--Seller:0.153519, Buyer:0.217693
Trial 4--Seller Return:1.057180, Buyer Return:-0.912673
Sample 100 is added.
Da

测试：

In [ ]:
tickers_eval = ['DELL', 'TSLA', 'QQQ', 'NET']
start_date, end_date = "2018-01-01", "2023-12-31"
stock_data_eval = []
for ticker in tickers_train:
    ticker_data = yf.Ticker(ticker).history(start=start_date, end=end_date)['Close']
    ticker_data = ticker_data/np.mean(ticker_data)*1000
    stock_data_eval.append(ticker_data)
stock_data_eval = np.array(stock_data_eval)
D = stock_data_eval.shape[0]
T, N, M = 1.0, 1000, 10000
dt = T/N

s0 = stock_data_eval[:,-1]
mu, sigma, rho = get_parameters(stock_data_eval, dt)
r = 0.04
option_func = partial(max_call_option, K = 1234)